<a href="https://colab.research.google.com/github/metrythm-AB/RAG-Implementation/blob/main/Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone-client
!pip install transformers sentence-transformers
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 20.9 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone, ServerlessSpec
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


In [ ]:
pc=Pinecone(api_key="da1ce8b8-1a1f-47a8-bb77-677c68a0a17f")
index_name='basic-rag'
pc.create_index(name=index_name,dimension=384, metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1"))


In [ ]:
embedder= SentenceTransformer('all-MiniLM-L6-v2')
gen_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
gen_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
gen_tokenizer.pad_token = gen_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
documents=[
    "Artificial Intelligence (AI) is a branch of computer science focused on creating systems capable of performing tasks that would typically require human intelligence. This field encompasses a range of capabilities, including learning, reasoning, problem-solving, perception, and language understanding."
    "The concept of AI has been around for centuries, but the formal field of AI research began in the 1950s. Key figures in the early development of AI include Alan Turing, who proposed the Turing Test as a measure of machine intelligence. In 1956, the term Artificial Intelligence was coined at the Dartmouth Conference, where researchers like John McCarthy, Marvin Minsky, Nathaniel Rochester, and Claude Shannon discussed the potential of creating machines that could simulate human intelligence."
    "Early AI research focused on symbolic reasoning and rule-based systems. These systems used predefined rules to make decisions and solve problems. However, the limitations of early AI were evident due to computational constraints and a lack of data."
    "AI research experienced periods of reduced funding and interest, known as AI winters, during the late 1970s and late 1980s. These periods were marked by high expectations that were not met due to the limitations of existing technology and the difficulty in scaling AI systems. However, advances in algorithms, increased computational power, and the availability of large datasets led to a resurgence in AI research and development in the 1990s."
    "The early 21st century saw significant advancements in AI, particularly with the rise of machine learning and deep learning. Machine learning, a subset of AI, involves training algorithms on data to enable them to learn and make predictions or decisions without explicit programming. Deep learning, a specialized form of machine learning, uses artificial neural networks to model complex patterns in data."
    "One of the most notable achievements in modern AI is the development of deep learning algorithms that have achieved state-of-the-art performance in various tasks, such as image recognition, speech recognition, and natural language processing. The success of these algorithms has been driven by improvements in computational power, particularly with the use of graphics processing units (GPUs) and cloud computing."
    "AI technologies have been integrated into various applications across multiple domains. Natural Language Processing (NLP) enables machines to understand, interpret, and generate human language. Applications include language translation, sentiment analysis, and chatbots. Technologies such as OpenAI's GPT-3 and BERT have made significant strides in NLP, providing more accurate and context-aware language models."
    "Computer vision involves enabling machines to interpret and analyze visual data from the world. This includes tasks such as object detection, image classification, and facial recognition. AI systems are used in autonomous vehicles, medical imaging, and surveillance."
    "Robotics combines AI with mechanical engineering to create intelligent machines capable of performing physical tasks. Applications include industrial robots, service robots, and robotic assistants. Advances in AI have improved the capabilities of robots to perform complex tasks and interact with humans."
    "AI algorithms are used to suggest products or content based on user preferences and behavior. Recommendation systems are widely used in e-commerce, streaming services, and social media platforms to provide personalized recommendations and enhance user experience."
    "In healthcare, AI is used for diagnostic purposes, personalized treatment plans, and drug discovery. AI algorithms analyze medical data to assist doctors in diagnosing diseases and predicting patient outcomes."
    "AI technologies are applied in finance for algorithmic trading, fraud detection, and risk management. AI systems analyze financial data to make trading decisions and detect anomalies in transactions."
    "The development and deployment of AI raise several ethical and societal issues. AI systems can perpetuate or even amplify existing biases if trained on biased data. Ensuring fairness and reducing bias in AI systems is a critical challenge. Researchers are working on developing methods to detect and mitigate bias in AI algorithms."
    "AI technologies often require large amounts of data, raising concerns about privacy and data protection. Ensuring that AI systems are used responsibly and that user data is handled securely is essential."
    "Questions about the control and decision-making of autonomous systems are important considerations. Ensuring that humans retain control over AI systems and that these systems operate within ethical boundaries is crucial."
    "The automation of tasks through AI may lead to job displacement and changes in the workforce. Addressing the impact of AI on employment and providing support for affected workers is an important consideration."
    "The future of AI includes ongoing research in several areas. General AI, or Artificial General Intelligence (AGI), aims to create machines with generalized cognitive abilities similar to human intelligence. AGI remains a long-term goal and presents significant technical and ethical challenges."
    "Improving the explainability and transparency of AI systems is essential for building trust and understanding how decisions are made. Research in this area focuses on developing methods to make AI algorithms more interpretable and accountable."
    "Ensuring that AI is developed and deployed ethically is a priority. This includes establishing guidelines and regulations for AI research and applications to address ethical concerns and promote responsible use."
    "The integration of AI into various domains will likely involve collaboration between humans and AI systems. Research is focused on developing interfaces and interactions that enable effective collaboration and maximize the benefits of AI technology."
    "Artificial Intelligence is a rapidly evolving field with significant potential to impact various aspects of society. As AI technologies continue to advance, addressing ethical considerations, ensuring fairness, and promoting responsible development will be crucial for maximizing the benefits of AI while minimizing potential risks."
]
def chunk_document(doc, chunk_size=128):
    return [doc[i:i + chunk_size] for i in range(0, len(doc), chunk_size)]


chunks = []
chunk_embeddings = []
for doc in documents:
    doc_chunks = chunk_document(doc)
    chunks.extend(doc_chunks)
    chunk_embeddings.extend(embedder.encode(doc_chunks))


In [ ]:

import pinecone
pc = pinecone.Pinecone(api_key="da1ce8b8-1a1f-47a8-bb77-677c68a0a17f", environment="us-east-1")
index_name = 'basic-rag'
index = pc.Index('basic-rag')
vectors = [(str(i), chunk_embeddings[i]) for i in range(len(chunk_embeddings))]
index.upsert(vectors)

{'upserted_count': 49}

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='vicgalle/gpt2-open-instruct-v1')
set_seed(42)

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def query_rag(query):

    query_embedding = embedder.encode(query).tolist()


    results = index.query(vector=query_embedding, top_k=3, include_values=True)


    retrieved_chunks = [chunks[int(match['id'])] for match in results['matches']]


    context = " ".join(retrieved_chunks)


    max_input_length = 1024 - len(gen_tokenizer.encode(query)) - 10
    context = context[:max_input_length]

    input_text = f"Ill provide context and query, answer this. Context: {context} Query: {query}"

    response = generator(input_text, max_length=1024, num_return_sequences=1, truncation=True)
    return response

    # inputs = gen_tokenizer.encode_plus(
    #     input_text,
    #     return_tensors="pt",
    #     padding=True,
    #     truncation=True,
    #     max_length=512,
    #     add_special_tokens=True
    # )

    # outputs = gen_model.generate(
    #    inputs['input_ids'],
    #    attention_mask=inputs['attention_mask'],
    #    max_length=512,
    #    num_return_sequences=1,
    #    pad_token_id=gen_tokenizer.eos_token_id
    # )

    #response = generator(input_text, max_length=512, num_return_sequences=1)

    # response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(response)
    #attention_mask = inputs['attention_mask']

In [ ]:
query = "What is the future of AI?"
response = query_rag(query)
print("Response:", response)

Response: [{'generated_text': 'Ill provide context and query, answer this. Context:  the benefits of AI technology.Artificial Intelligence is a rapidly evolving field with significant potential to impact various  aspects of society. As AI technologies continue to advance, addressing ethical considerations, ensuring fairness, and promoting  Artificial Intelligence (AI) is a branch of computer science focused on creating systems capable of performing tasks that would  Query: What is the future of AI? \nAI has the potential to revolutionize a wide range of industries such as healthcare, finance, healthcare, transportation, and transportation management, so it should be applied with caution when selecting technology. It should be used when it is appropriate and the process is not foolproof. AI has been shown to perform in a variety of scenarios in healthcare, education, employment, entertainment, manufacturing, retail, advertising, and a variety of other digital and scientific fields.\n\nA